In [1]:
from meta import DMMetaManager
meta_man = DMMetaManager(exam_tsv='metadata/exams_metadata.tsv', 
                         img_tsv='metadata/images_crosswalk.tsv', 
                         img_folder='preprocessedData/jpg_org', 
                         img_extension='jpg')

In [2]:
img_list, lab_list = meta_man.get_flatten_img_list()
len(img_list)

500

In [3]:
img_list[:5]

['preprocessedData/jpg_org/000135.jpg',
 'preprocessedData/jpg_org/000136.jpg',
 'preprocessedData/jpg_org/000137.jpg',
 'preprocessedData/jpg_org/000138.jpg',
 'preprocessedData/jpg_org/100151.jpg']

In [4]:
lab_list[:5]

[0, 0, 0, 0, 0]

In [5]:
from dm_image import DMImageDataGenerator
img_dat_gen = DMImageDataGenerator(rotation_range=90.)

Using Theano backend.


In [6]:
dat_gen = img_dat_gen.flow_from_img_list(
    img_list, lab_list, batch_size=10, seed=1,
    save_to_dir='./preprocessedData/jpg_batch',
    save_prefix='ran', save_format='jpg')

There are 34 cancer cases and 466 normal cases.


In [15]:
batch_d = dat_gen.next()

In [16]:
print type(batch_d)
print len(batch_d)

<type 'tuple'>
2


In [17]:
print batch_d[0].shape
print batch_d[1].shape

(10, 1, 256, 256)
(10,)


In [18]:
batch_d[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])